# LLM workflow examples

Anton Antonov  
[LLM::Functions at GitHub](https://github.com/antononcube/Raku-LLM-Functions)  
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com)   
May 2025

----

## Introduction

This notebook has examples of using Large Language Models (LLMs) for different _programmer_ tasks using Raku's LLM packages
["LLM::Functions"](https://raku.land/zef:antononcube/LLM::Functions) and ["LLM::Prompts"](https://raku.land/zef:antononcube/LLM::Prompts). 

**Remark:** Under the hood of "LLM::Functions" specialized LLM Web API services packages are used, like, ["WWW::OpenAI"](https://raku.land/zef:antononcube/WWW::OpenAI), ["WWW::Gemini"](https://raku.land/zef:antononcube/WWW::Gemini), ["WWW::LLaMA"](https://raku.land/zef:antononcube/WWW::LLaMA), and others. 

-----

## Setup

These packages are loaded by default if using ["Jupyter::Chatbook"](https://raku.land/zef:antononcube/Jupyter::Chatbook):

In [ ]:
#use LLM::Functions;
#use LLM::Prompts;
#use Text::SubParsers;
#use Data::TypeSystem;

Visualization package:

In [70]:
use JavaScript::Google::Charts;

Define LLM models access configurations:

In [45]:
my $conf4o = llm-configuration(‘chatgpt’, model => “gpt-4o”, max-tokens => 8192, temperature => 0.45);
my $conf41 = llm-configuration(‘chatgpt’, model => “gpt-4.1”, max-tokens => 8192, temperature => 0.45);
my $conf41-mini = llm-configuration(‘chatgpt’, model => “gpt-4.1-mini”, max-tokens => 8192, temperature => 0.45);

($conf4o, $conf41, $conf41-mini)».Hash».elems

(24 24 24)

-----

## Code translation

Make an LLM function that translates between programming languages:

In [4]:
my &fTrans = llm-function( -> $code, $lang = ‘Raku’ { llm-prompt(“CodeWriterX”)($lang) ~ “\n\n” ~ $code}, e => $conf4o)

-> **@args, *%args { #`(Block|3175731999944) ... }

Here is code (with Markdown code fences indicating the programming language):

In [6]:
my $code = q:to/END/;
```raku
100.rand xx 20
```
END

```raku
100.rand xx 20
```


Here is a translation:

In [7]:
say &fTrans($code, ‘Python’)

```python
import random

[random.uniform(0, 100) for _ in range(20)]
```


----

## Retrieving information for further computations

In this subsection we show to retrieve information into Raku array of hashmaps that can be further tabulated, queried, or visualized.

In [ ]:
my $res = llm-synthesize(
    [
        "Make a dataset of Brazil's states.",
        "Return an array of dictionaries, each dictionary with keys: State, Population, Area, GrossIncome2024.",
        llm-prompt('NothingElse')('JSON')
    ],
    e => $conf41-mini,
    form => sub-parser('JSON'):drop
);

deduce-type($res)

Vector(Struct([Area, GrossIncome2024, Population, State], [Int, Int, Int, Str]), 27)

Tabulate the result:

In [118]:
#% html
my @field-names = <State Population Area GrossIncome2024>;
$res ==> to-html(:@field-names)

State,Population,Area,GrossIncome2024
Acre,894470,164123,8500000000
Alagoas,3351543,27767,32000000000
Amapá,877613,142828,9000000000
Amazonas,4207714,1559159,60000000000
Bahia,14873064,564733,150000000000
Ceará,9132078,148920,110000000000
Distrito Federal,3055149,5779,140000000000
Espírito Santo,4064052,46077,90000000000
Goiás,7113540,340086,120000000000
Maranhão,7114598,331937,60000000000


Make a bubble chart:

In [119]:
#% html
my @dsBubbles = $res.kv.map( -> $i, %r { %( group => %r<State>, x => %r<Population>, y => %r<Area>, size => %r<GrossIncome2024> ) });
my @column-names = <group x y size>;

js-google-charts(
    'BubbleChart', 
    @dsBubbles, 
    :@column-names,
    title => 'Brazil states',
    hAxis => { title => 'Population'},
    vAxis => { title => 'Area'},
    div-id => 'BrazilBubbles', 
    format => 'html')

<!--Load the AJAX API-->
 
 
 

 
 <!--Div that will hold the pie chart-->

-----

## Vision: image description

AI / LLM vision can be done with the subs `llm-vision-synthesize` and `llm-vision-function`. For multi-modal the main functions `llm-synthesize` and `llm-function` can be used.

Here is an image URL:

In [71]:
#% markdown
my $url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/3/3e/Raccoon_in_Central_Park_%2835264%29.jpg/960px-Raccoon_in_Central_Park_%2835264%29.jpg';

"<img src=\"$url\" style=\"width:600px;\">"

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/3e/Raccoon_in_Central_Park_%2835264%29.jpg/960px-Raccoon_in_Central_Park_%2835264%29.jpg" style="width:600px;">

Using the specialized vision sub:

In [13]:
llm-vision-synthesize('Describe briefly what do you see in this image:', $url)

The image shows a raccoon standing on a forest floor. The raccoon has a bushy tail with black rings and a distinctive black mask around its eyes. The background includes rocks and trees, indicating a natural setting.

Another call using a (recent) multi-modal LLM:

In [14]:
llm-synthesize("Describe briefly what do you see in this image:\n $url", e => $conf41-mini)

The image shows a raccoon standing on a paved path in a park setting. The raccoon is looking directly at the camera, and there is greenery, including grass and trees, in the background. The scene appears to be taken during the day with natural light.

----
## Vision: reproducing a diagram

This subsection shows how the values of a certain statistics plot can be "read" and make a similar plot (via Raku.) 

In [61]:
#% chat html
How to specify the with of image link.

```html
<a href="https://example.com">
  <img src="image.jpg" alt="Description" style="width:200px;">
</a>
```

In [ ]:
#% markdown
my $urlBarChart = 'https://www.nationwidegroup.org/wp-content/uploads/2024/11/statistic_id1345543_us-consumers_-participation-by-shopping-event-during-thanksgiving-week-2020-2024.png';

"<img src=\"$urlBarChart\" style=\"width:600px;\">"


<img src="https://www.nationwidegroup.org/wp-content/uploads/2024/11/statistic_id1345543_us-consumers_-participation-by-shopping-event-during-thanksgiving-week-2020-2024.png" style="width:600px;">

Here we get a nested hashmap (dictionary) of the consumer data into different days and years:

In [68]:
my $res = llm-vision-synthesize([
    "For the given image make a JSON dictionary of the different years for the different days.",
    llm-prompt("NothingElse")("JSON")
    ],
    $urlBarChart,
    e => $conf41,
    form => sub-parser('JSON'):drop
);

deduce-type($res)

Vector(Pair(Atom((Str)), Assoc(Atom((Str)), Atom((Int)), 5)), 4)

Here we make a plot using ["JavaScript::Google::Charts"](https://raku.land/zef:antononcube/JavaScript::Google::Charts):

In [69]:
#% html
my @dsBar = $res.map({ [|$_.value.clone, Group => $_.key]})».Hash;

js-google-charts('BarChart',
        @dsBar,
        :!horizontal,
        backgroundColor => 'GhostWhite',
        bar => { groupWidth => '75%' },
        div-id => 'visionBars',
        format => 'html');

<!--Load the AJAX API-->
 
 
 

 
 <!--Div that will hold the pie chart-->

**Remark:** The order of the days and years is arbitrary due to Raku's non-ordered hashmaps.

-----

## References

### Articles

[AA1] Anton Antonov,
["Workflows with LLM functions"](https://rakuforprediction.wordpress.com/2023/08/01/workflows-with-llm-functions/),
(2023),
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com).


### Notebooks

[AAn1] Anton Antonov,
["Workflows with LLM functions (in Raku)"](https://community.wolfram.com/groups/-/m/t/2982320),
(2023),
[Wolfram Community](https://community.wolfram.com).

[AAn2] Anton Antonov,
["Workflows with LLM functions (in Python)"](https://community.wolfram.com/groups/-/m/t/3027081),
(2023),
[Wolfram Community](https://community.wolfram.com).

[AAn3] Anton Antonov,
["Workflows with LLM functions (in WL)"](https://community.wolfram.com/groups/-/m/t/3027081),
(2023),
[Wolfram Community](https://community.wolfram.com).


### Packages

[AAp1] Anton Antonov,
[LLM::Functions Raku package](https://github.com/antononcube/Raku-LLM-Functions),
(2023-2024),
[GitHub/antononcube](https://github.com/antononcube).
([raku.land](https://raku.land/zef:antononcube/LLM::Functions)).

[AAp2] Anton Antonov,
[LLM::Prompts Raku package](https://github.com/antononcube/Raku-LLM-Prompts),
(2023-2024),
[GitHub/antononcube](https://github.com/antononcube).
([raku.land](https://raku.land/zef:antononcube/LLM::Prompts)).

[AAp3] Anton Antonov,
[Jupyter::Chatbook Raku package](https://github.com/antononcube/Raku-Jupyter-Chatbook),
(2023-2024),
[GitHub/antononcube](https://github.com/antononcube).
([raku.land](https://raku.land/zef:antononcube/Jupyter::Chatbook)).

[AAp4] Anton Antonov,
[Jupyter::Chatbook Raku package](https://github.com/antononcube/Raku-JavaScript-Google-Charts),
(2024),
[GitHub/antononcube](https://github.com/antononcube).
([raku.land](https://raku.land/zef:antononcube/JavaScript::Google::Charts)).


### Videos

[AAv1] Anton Antonov,
["Robust LLM pipelines (Mathematica, Python, Raku)"](https://youtu.be/QOsVTCQZq_s),
(2024),
[YouTube/@AAA4Predictions](https://www.youtube.com/@AAA4Predictions).

[AAv2] Anton Antonov,
["Integrating Large Language Models with Raku"](https://www.youtube.com/watch?v=-OxKqRrQvh0),
(2023),
[The Raku Conference 2023 at YouTube](https://www.youtube.com/@therakuconference6823).